In [13]:
# --------- @property ---------
# https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work-in-python
# https://docs.python.org/3/howto/descriptor.html 
# allow you to define a method(), but can access it like an attribute

# Example 1
class Foo:
    def __init__(self, my_word):
        self._word = my_word
    @property
    def word(self):
        return self._word
# word() is now a attribute instead of a method
print(Foo('ok').word)     # ok

class Bar:
    def __init__(self, my_word):
        self._word = my_word
    def word(self):
        return self._word
print(Bar('ok').word())   # ok   # word() is a method

# Example 2
class Employee:
    def __init__(self, first, last) -> None:
        self.first = first
        self.last = last
    
    @property
    def email(self):
        return f'{self.first}.{self.last}@email.com'
    
    @property
    def fullname(self):
        return f'{self.first} {self.last}'
    
    @fullname.setter    # using fullname property, then call property's __setter__, hence fullname.setter
    def fullname(self, name):
        first, last = name.split(' ')
        self.first = first
        self.last = last
    
    @fullname.deleter    # property() also has __deleter__, __getter__
    def fullname(self):
        print('Delete Name!')
        self.first = None
        self.last = None

emp_1 = Employee('John', 'Smith')
print(emp_1.first)      # John
emp_1.fullname = 'Corey Schafer'
print(emp_1.fullname)   # Corey Schafer
print(emp_1.email)      # John.Smith@email.com     # email() is a attribute
del emp_1.fullname      # Delete Name!   # del keyword calls fullname.deleter
print(emp_1.first)      # None

ok
ok
John
Corey Schafer
Corey.Schafer@email.com
Delete Name!
None


In [32]:
# --------- getter, setter ---------
class Time:
    def __init__(self, hour=0, minute=0):
        self._hour = hour  # 0-23
        if minute < 0 or minute > 59:
            raise ValueError('minute must be 0-59')
        else:
            self.minute = minute

    @property       
    def hour(self):
        """getter hour."""
        return self._hour

    @hour.setter            
    def hour(self, hour):
        """setter hour."""
        if not (0 <= hour < 24): raise ValueError(f'Hour ({hour}) must be 0-23')
        self._hour = hour
    
    @property
    def minute(self):
        """getter minute."""
        return self._minute

    @minute.setter
    def minute(self, minute):
        """setter minute."""
        if not (0 <= minute < 60): raise ValueError(f'Minute ({minute}) must be 0-59')
        self._minute = minute

    def set_time(self, hour=0, minute=0):
        """setter hour, minute."""
        self.hour = hour
        self.minute = minute
        
    @property 
    def time(self):
        """Return hour, minute as a tuple."""
        return (self.hour, self.minute)

    @time.setter
    def time(self, time_tuple):
        """Set time from a tuple containing hour, minute."""
        self.set_time(time_tuple[0], time_tuple[1])

    def __repr__(self):
        """Return Time string for repr()."""
        return (f'Time(hour={self.hour}, minute={self.minute})')

    def __str__(self):
        """Return Time string in 12-hour clock format."""
        return (('12' if self.hour in (0, 12) else str(self.hour)) + 
                f':{self.minute:0>2}' + 
                (' AM' if self.hour < 12 else ' PM'))

wake_up = Time(hour=6, minute=30)
wake_up             # Time(hour=6, minute=30)      # calls __repr__, i.e. display "Time" object
print(wake_up)      # 6:30 AM       # calls __str__, i.e. display "Time" string
wake_up.hour        # 6             # calls property getter method, hour() is now an attribute, hence .hour
wake_up.set_time(hour=7, minute=45)    
print(wake_up)      # 7:45 AM
wake_up.hour = 5                    # Setting an attribute via property
print(wake_up)      # 5:30 AM 
# wake_up.hour = 45 # ValueError: Hour (45) must be 0-23
# However, you can bypass setter method by accessing directly the private attributes
wake_up._hour = 45                  # bypass setter method
print(wake_up)      # 45:30 AM             

wake_up2 = Time(hour=50, minute=30)
print(wake_up2)     # 50:30 AM
# wake_up3 = Time(hour=6, minute=70)  # ValueError: Minute (70) must be 0-59



6:30 AM
7:45 AM
5:45 AM
45:45 PM
50:30 PM


In [46]:
# --------- Private attributes ---------
# https://stackoverflow.com/questions/14671487/what-is-the-difference-in-python-attributes-with-underscore-in-front-and-back
# Naming convention:
    # _attribute: protected (not enforced)
    # __attribute: private  (enforced by Python name mangling)
# self.__private_data. Then Python automatically renames it as _ClassName__private_data (name mangling)
# so instance.__private_data will not access this attribute
# but instance._PrivateClass__private_data will still be able to access it

class PrivateClass:
    def __init__(self):
        """Initialize the public and private attributes."""
        self.public_data = "public"         # public attribute
        self.__private_data = "private"     # private attribute, preceded by __

instance = PrivateClass()
print(instance.public_data)                     # public

#instance.__private_data    # AttributeError: 'PrivateClass' object has no attribute '__private_data'

print(instance._PrivateClass__private_data)     # private
instance._PrivateClass__private_data = 'modified'
instance._PrivateClass__private_data            # modified

public
private


'modified'

In [45]:
# --------- Class variable ---------
# https://www.youtube.com/watch?v=BJ-VvGyQxho&list=PL-osiE80TeTsqhIuOqKhwlXsIBIdSeYtc&index=2 
class Foo:
    num_of_foos = 0     # class variable, shared by all instances
    raise_amt = 1.04    # class variable
    def __init__(self, x):
        self.instance_var = x
        Foo.num_of_foos += 1    # class safe way to use class variable
    def raise_me(self):
        # self.raise_amt allows object / subclass to override class variable
        return self.instance_var * self.raise_amt   

bar = Foo(1)
baz = Foo(1)
Foo.num_of_foos     # 2
bar.num_of_foos     # 2
bar.raise_me()      # 1.04
bar.raise_amt = 1.5 # create instance variable that overrides class variable
bar.raise_me()      # 1.5

2

In [49]:
# --------- Class method ---------
# accepts class instead of object as the first argument, i.e. cls, instead of self
# i.e. you don't need to create an instance to call class method
class Foo:
    raise_amt = 1.04    # class variable
    def __init__(self, x):
        self.instance_var = x
    def raise_me(self):
        return self.instance_var * self.raise_amt
    @classmethod
    def set_raise_amt(cls, amount):
        cls.raise_amt = amount 

bar = Foo(1)
Foo.set_raise_amt(1.5)      # class method
print(bar.raise_me())   # 1.5

# Alternative constructor with class method
class Employee:
    def __init__(self, first, last):
        self.first = first
        self.last = last
    @classmethod
    def from_string(cls, emp_str):  # alternative constructor
        first, last = emp_str.split(' ')
        return cls(first, last)    # cls() is the same as Employee()

emp_1 = Employee('John', 'Smith')
emp_2 = Employee.from_string('Jane Doe')

1.5


In [50]:
# --------- Static method ---------
# doesn't pass anything automatically, i.e. no self or cls
# use static method when you don't need to access class or instance
class Employee:
    @staticmethod
    def is_workday(day):
        if day.weekday() == 5 or day.weekday() == 6:
            return False
        return True

import datetime
my_date = datetime.date(2016, 7, 10)        # this is a Sunday
print(Employee.is_workday(my_date))     # False

False


In [48]:
# --------- Validate attributes ---------
# method 1: validation occurs in __init__
# see above examples

# method 2: validation function outside __init__
# Pros: can reuse checker function
# Cons: __init__ not clean 
# Cons: attribute can still be assigned to invalid value after the initialization without raising exception.
class Foo1:
    def __init__(self, x):
        self.__x = self._positive_checker(x)

    def _positive_checker(self, x):       #_positive_checker is a private fxn
        if x < 0:
            raise ValueError('x must be positive')
        else:
            return x

    @property
    def x(self):
        return self.__x

# num = Foo1(-2)    # ValueError: x must be positive
num = Foo1(5)
# num.x = 3         # AttributeError: can't set attribute 'x'
                         # x is private attribute, without setter method, i.e. cannot alter after init
num._Foo1__x = -2        # however, private attribute can be accessed if you know Python name mangling
num.x               # -2


-2

In [ ]:
# --------- Polymorphism ---------
import math
class Shape:
    def __init__(self, name):
        self.name = name
    def area(self):
        pass
    def __str__(self):
        return self.name

class Square(Shape):
    def __init__(self, length):
        super().__init__("Square")
        self.length = length
    def area(self):
        return self.length**2

class Circle(Shape):
    def __init__(self, radius):
        super().__init__("Circle")
        self.radius = radius
    def area(self):
        return math.pi * self.radius**2 

a = Square(4)
b = Circle(7)
for shape in (a, b):
    print(f'Area of {shape} is {shape.area():.2f}')
# Area of Square is 16.00
# Area of Circle is 153.94

In [24]:
# --------- Method overloading Singledispatch ---------
# https://martinheinz.dev/blog/50
# supported in Python standard library
# Singledispatch can only overload on the first argument


# Example 1: for methods in a class
from functools import singledispatchmethod, singledispatch
class Foo:
    @singledispatchmethod   # this is for method in a class, otherwise use @singledispatch
    def add(self, *args):
        res = 0
        for x in args:
            res += x
        print(res)
    @add.register(str)
    def _(self, *args):
        string = ' '.join(args)
        print(string)
    @add.register(list)
    def _(self, *args):
        myList = []
        for x in args:
            myList += x
        print(myList)

obj = Foo()
obj.add(1, 2, 3)        # 6
obj.add('I', 'love', 'Python')      # I love Python
obj.add([1, 2], [3, 4], [5, 6])     # [1, 2, 3, 4, 5, 6]

# Example 2: you can stack singledispatch
class Bar:
    @singledispatchmethod
    def output(self, arg):
        print(f"I'm a string: {arg}")
 
    @output.register(int)
    @output.register(float)
    def _(self, arg):
        print(f"I'm a number: {arg}")

bar = Bar()
bar.output('Hello')         # I'm a string: Hello
bar.output(5)               # I'm a number: 5
bar.output(2.5)             # I'm a number: 2.5

# Example 3: for independent methods
from datetime import date, time

@singledispatch
def format(arg):
    print(arg)

@format.register            # syntax version 1
def _(arg: date):
    print(f"{arg.day}-{arg.month}-{arg.year}")

@format.register(time)      # syntax version 2
def _(arg):
    print(f"{arg.hour}:{arg.minute}:{arg.second}")

format("today")                  # today
format(date(2021, 5, 26))        # 26-5-2021
format(time(19, 22, 15))         # 19:22:15

6
I love Python
[1, 2, 3, 4, 5, 6]
I'm a string: Hello
I'm a number: 5
I'm a number: 2.5
today
26-5-2021
19:22:15


In [33]:
# --------- Method overloading Multipledispatch ---------
# https://martinheinz.dev/blog/50
# not supported by standard library, must pip install multipledispatch
from multipledispatch import dispatch 
# example 1: proof of concept
class Foo:
    @dispatch(list, str)     # not dispatchmethod here
    def concatenate(a, b):
        a.append(b)
        return a

    @dispatch(str, str)
    def concatenate(a, b):   # have to use the same function name, not _
        return a + b

    @dispatch(str, int)
    def concatenate(a, b):
        return a + str(b)

foo = Foo()
print(foo.concatenate(["a", "b"], "c"))     # ['a', 'b', 'c']
print(foo.concatenate("Hello", "World"))    # HelloWorld
print(foo.concatenate("a", 1))              # a1

# example 2: union types - much more generic
@dispatch((list, tuple), (str, int))
def concatenate(a, b):
    return list(a) + [b]

print(concatenate(["a", "b"], "c"))     # ['a', 'b', 'c']
print(concatenate(("a", "b"), 1))       # ['a', 'b', 1]

# example 3: abstract types (like in Java) - even more generic
from collections.abc import Sequence
# Sequence is an abstract type, which means it is a type that is not meant to be instantiated, but rather inherited from.
# covers things like list, tuple, range, etc. but not dict
@dispatch(Sequence, (str, int))
def concatenate(a, b):
    return list(a) + [b]

['a', 'b', 'c']
HelloWorld
a1
['a', 'b', 'c']
['a', 'b', 1]


In [1]:
# --------- Operator overloading ---------
# Overload + and += operators
class Complex_number:
    def __init__(self, real, imaginary):
        self.real = real
        self.imaginary = imaginary

    def __add__(self, right):       # binary operators must provide 2 parameters
        return Complex_number(self.real + right.real, 
                       self.imaginary + right.imaginary)

    def __iadd__(self, right):
        """Overrides the += operator."""
        self.real += right.real
        self.imaginary += right.imaginary
        return self

    def __repr__(self):
        return (f'({self.real}' + 
                (' + ' if self.imaginary >= 0 else ' - ') +
                f'{abs(self.imaginary)}i)')

x = Complex_number(real = 2, imaginary = 4)
x       # (2 + 4i)
y = Complex_number(real = 5, imaginary = -1)
y       # (5 - 1i)
x + y   # (7 + 3i)
x += y
x       # (7 + 3i)
y       # (5 - 1i)

(5 - 1i)